In [ ]:
import random 
from numpy import zeros, sqrt, pi, exp,linspace, floor
import pylab 
from statistics import NormalDist
from scipy.optimize import curve_fit
L = 1000
N = 1000
iter = 10000
stats = zeros([2*N, iter], dtype = int)
Nlist = [n for n in range(N)]
Ncum = []
NcumIt = [0]
Ncumc = [N]
history = zeros([2*N,iter], dtype = int)

In [ ]:
Li = []
for i in range(1,L):
    for j in range(1,L):
        Li.append((i,j))

for n in range(N):
    a = random.choice(Li)
    history[n,0], history[n+N,0] = a[0], a[1]
    Li.remove(a)


In [ ]:
for iterations in range(1,iter):
    space = zeros([L+1,L+1], dtype = int)
    for j in Nlist:
        a = space[history[j,iterations-1], history[j+N, iterations-1]]
        if a>0:
            Nlist.remove(j)
            Ncum.append(j)
            for q in range(iterations, iter):
                history[j,q], history[j+N,q] = history[j,iterations-1], history[j+N,iterations-1]
            for i in Nlist:
                if history[j,iterations-1] == history[i,iterations-1] and history[j+N,iterations-1] == history[i+N,iterations-1]:
                    Nlist.remove(i)
                    for q in range(iterations, iter):
                        history[i,q], history[i+N,q] = history[i,iterations-1], history[i+N,iterations-1]
                    Ncum.append(i)
 
            

        if not a>0:
            space[history[j,iterations-1], history[j+N, iterations-1]]+=1

    NcumIt.append(iterations)
    Ncumc.append(len(Nlist))

    for n in Nlist:
        dir = random.randint(1,4)
        if dir == 1:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1]+1, history[n+N,iterations-1]
            if history[n,iterations]>L:
                history[n,iterations]-=2

            

        if dir == 2:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1]-1, history[n+N,iterations-1]

            if history[n,iterations]<0:
                history[n,iterations]+=2

        if dir == 3:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1], history[n+N,iterations-1]+1

            if history[n+N,iterations]>L:
                history[n+N,iterations]-=2


        if dir == 4:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1], history[n+N,iterations-1]-1


            if history[n+N,iterations]<0:
                history[n+N,iterations]+=2

    for n in range(N):
        stats[n,iterations], stats[n+N, iterations] = int(history[n,iterations]-history[n,0]), int(history[n+N,iterations]-history[n+N,0])


In [ ]:
%matplotlib notebook
statboi = True
if statboi == True:
    fig = pylab.figure(figsize=(7,7))
    Totx = []
    Toty = []
    ax = pylab.subplot(111)
    for n in (Nlist + Ncum):
        for j in range(iter):
            Totx.append(stats[n,j])
            Toty.append(stats[n+N,j])
    sumT = Totx+Toty
    x,bins,p=ax.hist(sumT, bins = (max(sumT) - min(sumT)), density = True, alpha = 0.45, linewidth=3, color = "navy")
    norm = NormalDist.from_samples(sumT)
    mu, sigma = norm.mean, norm.stdev 
    print(mu, sigma)
    sigmap = sigma/sqrt(len(bins))
    print(sigma/sqrt(len(bins)))
    def f(x):
        return 1/(sigma*sqrt(2*pi))*exp(-1/2*((x-mu)/sigma)**2)
    xspace = linspace(min(sumT), max(sumT), 1000)
    ax.plot(xspace, [f(i) for i in xspace], "r--", lw = 3)
    ax.plot([mu,mu], [0,1/(sigma*sqrt(2*pi))], color = "black", lw = 3)
    ax.grid(True, ls= "--", color ="black", alpha = 0.3)
    ax.set_title("Histogram of step deviation from original position for ALL specimen. N = " + str(N))
    pylab.legend(["Gaussian fit", "Center of Gaussian fit at "+ r"$\mu =$" + str(floor(100*mu)/100),"Histogram data"], title = "Legend:")
    ax.set_xlabel("Step deviation from starting point")
    ax.set_ylabel("Probability density")
pylab.show()

In [ ]:
%matplotlib notebook
statboi = True
if statboi == True:
    fig = pylab.figure(figsize=(10,10))
    Totx = []
    Toty = []
    ax = pylab.subplot(111)
    for n in Nlist:
        for j in range(iter):
            Totx.append(stats[n,j])
            Toty.append(stats[n+N,j])
    sumT = Totx+Toty
    x,bins,p=ax.hist(sumT, bins = (max(sumT) - min(sumT)), density = True, alpha = 0.45, linewidth=3, color = "navy")
    norm = NormalDist.from_samples(sumT)
    mu, sigma = norm.mean, norm.stdev 
    print(mu, sigma)
    sigmap = sigma/sqrt(len(bins))
    print(sigma/sqrt(len(bins)))
    def f(x):
        return 1/(sigma*sqrt(2*pi))*exp(-1/2*((x-mu)/sigma)**2)
    xspace = linspace(min(sumT), max(sumT), 1000)
    ax.plot(xspace, [f(i) for i in xspace], "r--", lw = 3)
    ax.plot([mu,mu], [0,1/(sigma*sqrt(2*pi))], color = "black", lw = 3)
    ax.grid(True, ls= "--", color ="black", alpha = 0.3)
    ax.set_title("Histogram of step deviation from original position for alive specimen. N = " + str(len(Nlist)))
    pylab.legend(["Gaussian fit", "Center of Gaussian fit at "+ r"$\mu =$" + str(floor(100*mu)/100),"Histogram data"], title = "Legend:")
    ax.set_xlabel("Step deviation from starting point")
    ax.set_ylabel("Probability density")
pylab.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

from scipy.stats import multivariate_normal
from numpy import meshgrid

X = linspace(min(Totx),max(Totx),100)

Y = linspace(min(Toty),max(Toty),100)

X, Y = meshgrid(X,Y)

normx = NormalDist.from_samples(Totx)
mux, sigmax = normx.mean, normx.stdev 

normy = NormalDist.from_samples(Toty)
muy, sigmay = normy.mean, normy.stdev 


pos = zeros(X.shape+(2,))

pos[:,:,0]=X

pos[:,:,1]=Y

rv = multivariate_normal([mux, muy],[[sigmax, 0], [0, sigmay]])

fig = pylab.figure(figsize=(7,7))

ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("Step deviation on x axis of " + r"$\Omega$")
ax.set_ylabel("Step deviation on  y axis of " + r"$\Omega$")
ax.set_zlabel("Probability density")
ax.set_title("Combined Gaussian distribution for each spacial domain")

im = ax.plot_surface(X, Y, rv.pdf(pos), cmap="plasma")
pylab.colorbar(im, shrink = 0.5)
pylab.show()

In [ ]:
if True==True:
    fig = pylab.figure(figsize = (7,7))
    ax = pylab.subplot(111)
    for n in Ncum:
        ax.plot(history[n], history[n+N], "black", alpha = 0.3)
    for n in  Nlist:
        ax.plot(history[n], history[n+N])
    ax.grid(True, ls = "--", color = "black", alpha = 0.5)
    ax.set_title("Motion of vicious walkers in " + r"$\Omega$")

    
    ax.set_xlabel("x position in " + r"$\Omega$")
    ax.set_ylabel("y position in " + r"$\Omega$")
    ax.set_xlim([0,L])
    ax.set_ylim([0,L])
pylab.show()

In [ ]:
if True == True:
    fig = pylab.figure(figsize = (7,7))
    def invexp(t,a,b):
        return a*exp(-b*t)


    timespace = linspace(0,iterations)
    timeline = NcumIt + [iterations]
    particleC = Ncumc + [Ncumc[-1]]
    
    prop, _ = curve_fit(invexp, timeline, particleC)
    alf,bet = prop
    
    pylab.plot(timespace, invexp(timespace,alf,bet), color = "navy", lw = 3)
    pylab.plot(timeline,particleC, ls = "--", color = "red", lw = 3)
    for i in range(0,len(timeline),50):
        pylab.plot([timeline[i],timeline[i]], [particleC[i], invexp(timeline[i],alf,bet)], "--", color = "black", alpha = 0.3)
    pylab.legend([ "Exponential fit of form:" + r"$N_0 e^{-bt}$", "particle count", "Error"],title = "Legend:" )
    pylab.plot()
    pylab.grid(True, ls= "--", color ="black", alpha = 0.3)
    pylab.title("Alive walkers over iteration count")
    pylab.xlabel("Iteration count [t]")
    pylab.ylabel("Walker count [N]")
print(alf,bet)
pylab.show()